In [ ]:
!pip install pyvi
!pip install unidecode
!pip install emoji

     |████████████████████████████████| 8.5MB 902kB/s eta 0:00:01
     |████████████████████████████████| 440kB 43.5MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/40/fa/ee/3afb15958ad26f3aef88d61c316b4d1af8a97660aa24e6e6d7
Successfully built python-crfsuite


In [ ]:
#Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

#Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.pooling import GlobalMaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.layers import *
from keras import backend
from sklearn.metrics import f1_score, confusion_matrix
import tensorflow as tf

from pyvi import ViTokenizer
from pyvi import ViUtils

Using TensorFlow backend.


### Importing the dataset
***

The dataset: 'data1.csv' is read and loaded as pandas dataframe.  
Let's have a look at the data

In [ ]:
#Importing the dataset
dataset = pd.read_csv('/kaggle/input/cleandata/data1.csv')
df = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/test.csv')
dataset.head()

,Comment,Rating
0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",1
1,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0
2,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",1
3,Em có đọc review thấy mng bảo trà sữa nướng đề...,0
4,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",1


In [ ]:
def lowerCase(data):
    data = data.lower()
    return data
def remove_punctuations(data):
    punct_tag = re.compile(r'[^\w\s]')
    data = punct_tag.sub(r'',data)
    return data
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data
def remove_n(data):
    html_tag=re.compile(r'\n')
    data=html_tag.sub(r' ',data)
    return data
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data
def remove_x(data):
    html_tag=re.compile(r'\xa0')
    data=html_tag.sub(r' ',data)
    return data

In [ ]:
dataset['Comment'] = dataset['Comment'].apply(lambda z: lowerCase(str(z)))

dataset['Comment'] = dataset['Comment'].apply(lambda z: remove_punctuations(z))

dataset['Comment'] = dataset['Comment'].apply(lambda z: remove_n(z))

dataset['Comment'] =  dataset['Comment'].apply(lambda z: remove_x(z))

dataset['Comment'] =  dataset['Comment'].apply(lambda z: remove_html(z))

dataset['Comment'] =  dataset['Comment'].apply(lambda z: remove_url(z))

dataset['Comment'] =  dataset['Comment'].apply(lambda z: remove_emoji(z))

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39071 entries, 0 to 39070
Data columns (total 2 columns):
Comment    39071 non-null object
Rating     39071 non-null int64
dtypes: int64(1), object(1)
memory usage: 610.6+ KB


In [ ]:
X_train = dataset['Comment'].values
y_train = dataset['Rating'].values
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
print(X_train[:2])
y_train[:2]

['xôi dẻo đồ ăn đậm vị hộp xôi được lót lá trông rất thích'
 'gọi ship 1 xuất cari gà bánh naan và 3 miếng gà nướngđược tặng 1 coca đồ ăn khá ngon tổng 210k được giảm 50k còn 160k tuy nhiên gọi 3 miếng gà thì thiếu 1 miếng mà kể cả đó đủ ba miếng thì khẩu phần vẫn là quá ít so với giá 120k 1 suất']


array([1, 0])

### Text Preprocessing
***
Preprocessing the text so as to have a better data for our model.  
It comprises of steps such as removing non-ASCII characters, removing HTML tags, converting to lower-case, lemmatizing.

In [ ]:

def clean_text(X,y):
    idx = 0
    y_train = []
    processed = []
    for text in X:
        text = list(tf.keras.preprocessing.text.text_to_word_sequence(text))
        text = " ".join(text)
        input_text_pre_no_accent = str(ViUtils.remove_accents(text).decode("utf-8"))
        input_text_pre_accent = ViTokenizer.tokenize(text)
        input_text_pre_no_accent = ViTokenizer.tokenize(input_text_pre_no_accent)
        processed.append(input_text_pre_accent)
        processed.append(input_text_pre_no_accent)
        y_train.append(y[idx])
        y_train.append(y[idx])
        idx += 1
    return processed,y_train

Preprocessing the Training Set and Test set

In [ ]:
y_train[0]

1

In [ ]:
X_train_final,y_train = clean_text(X_train,y_train)
# X_test_final,y_test = clean_text(X_test,y_test)

In [ ]:
len(X_train_final)

78142

In [ ]:
print(X_train_final[:2])
print(y_train[:2])

['xôi dẻo đồ ăn_đậm vị hộp xôi được lót lá trông rất thích', 'xoi deo do an dam vi hop xoi duoc lot la trong rat thich']
[1, 1]


### Attention Layer
***

The basic concept of attention is that not all words contribute equally to the meaning of a sentence. Hence, their contribution must be weighted.  
How attention works is, it basically extracts words that are important to the meaning of the sentence and aggregate the representation of those informative words to form a sentence vector.

In [ ]:
# Attention Layer
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

Paramenter needed  


In [ ]:
#Tokenization and Padding
vocab_size = 60000
maxlen = 250
encode_dim = 20
batch_size = 32
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_final)
tokenized_word_list = tokenizer.texts_to_sequences(X_train_final)
X_train_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')

In [ ]:
print(np.shape(X_train_padded))
print(np.shape(X_train_final))

(78142, 250)
(78142,)


**EarlyStopping**  
It can be used to prevent overfitting.It basically waits a few epochs (5), monitoring the loss for the validation dataset.If the loss doesn't decrease for 2 epochs, it stops the training process.

**ModelCheckpoint**  
It is used for saving the best model during training. After each epoch, it takes a look at the Validation accuracy, if it improves globally, this is the best model we have seen till now during the training process and hence, saves it.

In [ ]:
#EarlyStopping and ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('model_best.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

### Building the Model
***
The model used comprises of BiDirectional LSTM with Attention layer on top of it, followed by a dense layer and finally a dense layer with sigmoid activation function to get the sentiment or the class.  
Optimiser used is ADAM

In [ ]:
#Building the model
model = Sequential()
embed = Embedding(input_dim = vocab_size, output_dim = 20, input_length = X_train_padded.shape[1], dropout = 0.1) 
model.add(embed)
model.add(Bidirectional(CuDNNLSTM(200, return_sequences = True)))
model.add(Dropout(0.1))
# model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
# model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Dropout(0.1))
model.add(AttentionWithContext())
model.add(Dropout(0.1))
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(256))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)

### Testing
***
Converting the test data into sequences of integers and padding them.  
Loading the best model and calculating the accuracy

In [ ]:
from keras.models import load_model
modelload = load_model('/kaggle/input/best-model/model_best (1).h5', custom_objects = {"AttentionWithContext" : AttentionWithContext, "backend" : backend})

In [ ]:

# df = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/test.csv')
df['Comment'] = df['Comment'].apply(lambda z: lowerCase(str(z)))

df['Comment'] = df['Comment'].apply(lambda z: remove_punctuations(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_n(z))

df['Comment'] = df['Comment'].apply(lambda z: remove_x(z))

df['Comment'] =  df['Comment'].apply(lambda z: remove_html(z))

df['Comment'] =  df['Comment'].apply(lambda z: remove_url(z))

df['Comment'] =  df['Comment'].apply(lambda z: remove_emoji(z))

data_test = pd.DataFrame({'input':df['Comment'],'id':df["RevId"]})

X_test = data_test['input'].values


In [ ]:
def clean_text_test(X):
    processed = []
    for text in X:
        text = list(tf.keras.preprocessing.text.text_to_word_sequence(str(text)))
        text = " ".join(text)
        input_text_pre_no_accent = str(ViUtils.remove_accents(text).decode("utf-8"))
        input_text_pre_accent = ViTokenizer.tokenize(text)
        processed.append(input_text_pre_accent)
    return processed

X_test_final = clean_text_test(X_test)
tokenized_word_list = tokenizer.texts_to_sequences(X_test_final)
X_test_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')

In [ ]:
y_pred = modelload.predict(X_test_padded)
print(y_pred[:10])
print(np.shape(y_pred))

(5103, 1)


In [ ]:
print(X_test_final[9])

In [ ]:
print(X_test_final[12])

In [ ]:
print(X_test_final[1])

In [ ]:
my_submission = pd.DataFrame({'RevId': np.array(df["RevId"]), 'Rating': np.array(y_pred).reshape(5103)})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission[:5]

,RevId,Rating
0,781115,0.038603
1,1219481,0.272514
2,1703765,0.990643
3,4870346,0.110394
4,2638711,0.981124
